<a href="https://colab.research.google.com/github/aSafarpoor/KGCLG/blob/main/LightGCN_part1_view0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from tqdm import tqdm 
import pandas as pd
import pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/KGCLG

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/KGCLG


In [3]:
!ls 

brands1.pickle			     meta_justV.rar
catedges1.pickle		     meta.pickle
iinits.pickle			     n2brands.csv
Industrial_and_Scientific.json	     uinits.pickle
meta128_2.csv			     ui.pickle
meta128.pickle			     view0_edges_dict.csv
meta_Industrial_and_Scientific.json  view0-embeddings.csv
meta_justV


In [4]:
# with open('ui.pickle', 'wb') as handle:
    # pickle.dump(ui, handle)
def pickle_reader(name):
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
        return b

In [5]:
'''
#should to be read from hard drive
* brands1.pickle			     
* catedges1.pickle
* meta128.pickle
* ui.pickle
'''
brandedges = pickle_reader("brands1.pickle")
catedges = pickle_reader("catedges1.pickle")
meta = pickle_reader("meta128.pickle")
ui = pickle_reader("ui.pickle")

In [6]:
# meta.to_csv('meta128_2.csv')

In [7]:
ne = {}
users = list(set(list(ui.reviewerID)))
items = list(set(list(ui.asin)))
for i in tqdm(items):
    ne[i] = np.array(meta[meta.asin==i].text2vec)
for i in tqdm(users):
        ne[i] = np.zeros(128) 

100%|██████████| 914150/914150 [00:01<00:00, 541665.10it/s]


In [8]:
len(users),len(items)

(914150, 131400)

In [9]:
c = 0
for i in ne[list(ne.keys())[0]][0]:
    c+=1
    if c%8==0:
        print()
    print(i,end=" ")  

-0.49363160133361816 1.3619556427001953 -0.2179161012172699 -6.23615026473999 0.8290451765060425 0.023235492408275604 0.2614201605319977 
0.39915648102760315 -0.41465768218040466 -0.08145319670438766 -0.7472809553146362 2.651082992553711 1.853583574295044 0.629177451133728 -1.4639167785644531 
-0.7146155834197998 0.5188632011413574 -1.5732369422912598 -0.6443413496017456 0.019265569746494293 -0.28444916009902954 -0.8464210033416748 -0.7730132341384888 
-0.7086719274520874 -0.08558221161365509 -1.574950933456421 2.605844259262085 -0.2358691394329071 2.4207653999328613 0.46508535742759705 -0.6952546238899231 
0.8865736722946167 0.950329065322876 -0.2409970760345459 0.23987874388694763 0.7404417991638184 1.1582133769989014 0.7801578044891357 -0.787824273109436 
0.5453243255615234 -1.914860486984253 0.16476848721504211 2.5813989639282227 0.8243767023086548 0.8499420881271362 -0.22687473893165588 0.4986346960067749 
0.44343286752700806 0.041543424129486084 0.6063492298126221 -0.108100585639

In [10]:
nvd = {}
cd = {}

for n in tqdm(ne):
    nvd[n]=[]
    cd[n]=0


100%|██████████| 1045550/1045550 [00:02<00:00, 445500.50it/s]


In [11]:
# create_node_vicinity from ui:
for ind in tqdm(range(len(ui))):
    row = ui.iloc[ind]
    e1, e2 = row['asin'],row['reviewerID']
    cd[e2] += 1    
    nvd[e2].append(e1)

100%|██████████| 1267323/1267323 [01:16<00:00, 16497.69it/s]


In [12]:
for u in ne:
    if len(ne[u])==1:
        ne[u]=ne[u][0]

for u in tqdm(nvd):
    vs = nvd[u];
    if len(vs)>0:
        for v in vs:
             ne[u] += ne[v]
        ne[u] /= cd[u] 
    else:
        pass

100%|██████████| 1045550/1045550 [05:36<00:00, 3108.55it/s]


In [13]:
type(ne)

dict

In [14]:
c = 0
for i in ne[v]:
    c+=1
    if c%8==0:
        print()
    print(i,end=' ')

-4.485982418060303 -0.260276198387146 1.212784767150879 3.1332290172576904 -1.9621391296386719 0.4387134909629822 0.456303209066391 
-1.734360694885254 -0.8565735816955566 -0.09660903364419937 0.9611631035804749 0.2548512816429138 0.24835483729839325 -1.7904777526855469 -0.969312310218811 
-0.06239697337150574 1.8894402980804443 2.692603588104248 -0.8404965996742249 -0.07570913434028625 1.0634002685546875 -0.7207652926445007 0.06705678254365921 
0.4888836145401001 -0.6563516855239868 0.0251675546169281 0.3054383099079132 -0.5016584396362305 0.04032491147518158 -0.30225294828414917 -0.0643211156129837 
0.1994936168193817 -0.0331849679350853 -0.10770058631896973 0.015035301446914673 0.26658856868743896 0.440921425819397 -0.22083188593387604 -0.005706608295440674 
-1.2028530836105347 -0.20822620391845703 -0.011241104453802109 -0.49025997519493103 0.6183903813362122 0.3173961937427521 0.7772966623306274 -0.4370029866695404 
-0.47990086674690247 -0.016778960824012756 0.614074170589447 -0.45

In [15]:
c = 0
for i in ne[u]:
    c+=1
    if c%8==0:
        print()
    print(i,end=' ')

-4.485982418060303 -0.260276198387146 1.212784767150879 3.1332290172576904 -1.9621391296386719 0.4387134909629822 0.456303209066391 
-1.734360694885254 -0.8565735816955566 -0.09660903364419937 0.9611631035804749 0.2548512816429138 0.24835483729839325 -1.7904777526855469 -0.969312310218811 
-0.06239697337150574 1.8894402980804443 2.692603588104248 -0.8404965996742249 -0.07570913434028625 1.0634002685546875 -0.7207652926445007 0.06705678254365921 
0.4888836145401001 -0.6563516855239868 0.0251675546169281 0.3054383099079132 -0.5016584396362305 0.04032491147518158 -0.30225294828414917 -0.0643211156129837 
0.1994936168193817 -0.0331849679350853 -0.10770058631896973 0.015035301446914673 0.26658856868743896 0.440921425819397 -0.22083188593387604 -0.005706608295440674 
-1.2028530836105347 -0.20822620391845703 -0.011241104453802109 -0.49025997519493103 0.6183903813362122 0.3173961937427521 0.7772966623306274 -0.4370029866695404 
-0.47990086674690247 -0.016778960824012756 0.614074170589447 -0.45

In [16]:
type(ne[v]),type(ne[u])

(pandas.core.series.Series, pandas.core.series.Series)

In [17]:
d2 = [[],[]]

In [18]:
for i in ne:
    d2[0].append(i)
    d2[1].append(ne[i])    

In [19]:
d3 = np.array(d2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [20]:
d3.shape

(2, 1045550)

In [21]:
d4 = d3.T

In [22]:
d4.shape

(1045550, 2)

In [23]:
d4[0][0]

'B00VIXNYC4'

In [24]:
c = 0
for i in d4[0][1]:
    c+=1
    if c%8==0:
        print()
    print(i,end=' ')

-0.49363160133361816 1.3619556427001953 -0.2179161012172699 -6.23615026473999 0.8290451765060425 0.023235492408275604 0.2614201605319977 
0.39915648102760315 -0.41465768218040466 -0.08145319670438766 -0.7472809553146362 2.651082992553711 1.853583574295044 0.629177451133728 -1.4639167785644531 
-0.7146155834197998 0.5188632011413574 -1.5732369422912598 -0.6443413496017456 0.019265569746494293 -0.28444916009902954 -0.8464210033416748 -0.7730132341384888 
-0.7086719274520874 -0.08558221161365509 -1.574950933456421 2.605844259262085 -0.2358691394329071 2.4207653999328613 0.46508535742759705 -0.6952546238899231 
0.8865736722946167 0.950329065322876 -0.2409970760345459 0.23987874388694763 0.7404417991638184 1.1582133769989014 0.7801578044891357 -0.787824273109436 
0.5453243255615234 -1.914860486984253 0.16476848721504211 2.5813989639282227 0.8243767023086548 0.8499420881271362 -0.22687473893165588 0.4986346960067749 
0.44343286752700806 0.041543424129486084 0.6063492298126221 -0.108100585639

In [25]:
df = pd.DataFrame (d4, columns = ['name','vector'])

In [30]:
c = 0
for i in df.iloc[10]['vector']:
    c+=1
    if c%8==0:
        print()
    print(i,end=' ')

-3.8787434101104736 2.355062961578369 3.0169825553894043 2.944418430328369 1.2378122806549072 2.0318517684936523 -1.3967607021331787 
1.4458060264587402 0.8918731212615967 0.8492927551269531 1.1267766952514648 1.706816554069519 0.9447339773178101 -2.198411464691162 1.7682394981384277 
1.3566417694091797 0.07312034070491791 0.6027237772941589 1.0025678873062134 -0.1985812932252884 1.2610926628112793 -0.1528639942407608 0.008250430226325989 
1.697692632675171 -0.6805959343910217 -1.0696098804473877 -0.06949396431446075 -0.9564684629440308 -0.1615409255027771 1.1864631175994873 0.43820464611053467 
-0.046108707785606384 0.7218206524848938 -0.3983442783355713 1.5170578956604004 0.046072155237197876 -0.7862252593040466 0.9624835252761841 -0.4781399071216583 
-0.5613325238227844 0.6050799489021301 0.3320158123970032 0.24279847741127014 -0.3798513412475586 0.42498070001602173 -0.4875723123550415 0.21997591853141785 
0.5642580986022949 -0.7676387429237366 -0.5576684474945068 0.0174017921090126

In [28]:
# df.to_csv('view0-embeddings.csv')

In [31]:
# write
df.to_json('view0-embeddings.json', orient = 'split', compression = 'infer', index = 'true')
 
# reading
# df = pd.read_json('file.json', orient ='split', compression = 'infer')

In [32]:
!ls -l --block-size=M

total 4177M
-rw------- 1 root root   22M Nov  3 20:03 brands1.pickle
-rw------- 1 root root   22M Nov  3 19:52 catedges1.pickle
-rw------- 1 root root  117M Nov  4 16:38 iinits.pickle
-rw------- 1 root root  741M Oct 18 20:35 Industrial_and_Scientific.json
-rw------- 1 root root   56M Nov  5 15:29 meta128_2.csv
-rw------- 1 root root  161M Nov  4 15:53 meta128.pickle
-rw------- 1 root root  608M Oct 18 20:15 meta_Industrial_and_Scientific.json
drwx------ 2 root root    1M Nov  4 14:28 meta_justV
-rw------- 1 root root  352M Nov  4 11:49 meta_justV.rar
-rw------- 1 root root  115M Nov  3 19:50 meta.pickle
-rw------- 1 root root   22M Nov  6 14:28 n2brands.csv
-rw------- 1 root root  101M Nov  4 16:38 uinits.pickle
-rw------- 1 root root   63M Nov  3 17:12 ui.pickle
-rw------- 1 root root   56M Nov  6 16:15 view0_edges_dict.csv
-rw------- 1 root root   10M Nov  6 17:11 view0-embeddings.csv
-rw------- 1 root root 1737M Nov  6 17:15 view0-embeddings.json


In [1]:
import pandas as pd

In [5]:
df2 = pd.read_json('view0-embeddings.json', orient ='split', compression = 'infer')